In [2]:
import os
import numpy as np
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

##Code for analysis of fMRI experiment

In [3]:
%matplotlib inline

In [6]:
#Read in data
data_dir = '/Users/ianballard/Dropbox/fd/'
rt_ser = pd.read_csv(data_dir + 'rt_ser.csv')
rt_switch_ser = pd.read_csv(data_dir + 'rt_switch_ser.csv')
rt_sim = pd.read_csv(data_dir + 'rt_sim.csv')
rt_switch_sim = pd.read_csv(data_dir + 'rt_switch_sim.csv')

In [29]:
rew_trial_1 = [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
trial_order_1 = [1, 2, 10, 6, 10, 6, 10, 4, 5, 10, 1, 2, 10, 3, 10, 1, 2, 10, 1, 2, 10, 1, 2, 10, 4, 5, 10, 4, 5, 10, 3, 10, 1, 2, 10, 4, 5, 10, 4, 5, 10, 6, 10, 3, 10, 4, 5, 10, 6, 10, 3, 10, 3, 10, 6, 10, 4, 5, 10, 3, 10, 6, 10, 1, 2, 10, 3, 10, 1, 2, 10, 4, 5, 10, 4, 5, 10, 6, 10, 3, 10, 6, 10, 1, 2, 10, 3, 10, 6, 10, 3, 10, 4, 5, 10, 6, 10, 1, 2, 10]
            
rew_trial_2 = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
trial_order_2 = [4, 5, 10, 3, 10, 4, 5, 10, 4, 5, 10, 1, 2, 10, 3, 10, 3, 10, 4, 5, 10, 4, 5, 10, 1, 2, 10, 6, 10, 1, 2, 10, 6, 10, 6, 10, 4, 5, 10, 4, 5, 10, 1, 2, 10, 3, 10, 6, 10, 6, 10, 6, 10, 4, 5, 10, 3, 10, 1, 2, 10, 6, 10, 1, 2, 10, 1, 2, 10, 1, 2, 10, 3, 10, 4, 5, 10, 1, 2, 10, 6, 10, 3, 10, 3, 10, 3, 10, 6, 10, 4, 5, 10, 3, 10, 6, 10, 1, 2, 10]
                
rew_trial_3 = [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
trial_order_3 = [1, 2, 10, 6, 10, 6, 10, 3, 10, 3, 10, 6, 10, 1, 2, 10, 1, 2, 10, 3, 10, 3, 10, 4, 5, 10, 4, 5, 10, 1, 2, 10, 6, 10, 3, 10, 6, 10, 4, 5, 10, 4, 5, 10, 4, 5, 10, 3, 10, 1, 2, 10, 4, 5, 10, 4, 5, 10, 6, 10, 3, 10, 4, 5, 10, 1, 2, 10, 1, 2, 10, 1, 2, 10, 4, 5, 10, 6, 10, 3, 10, 1, 2, 10, 3, 10, 6, 10, 6, 10, 3, 10, 4, 5, 10, 1, 2, 10, 6, 10]

In [30]:
def perform_RL(trial_order,rew_trial,alpha):
    V = {'b_plus':[0], 'b_minus' : [0], 'c_plus' : [0],'c_minus':[0]}
    for n,cond in enumerate(trial_order):
        if cond == 1: #A
            continue
        elif cond == 2:
            trial_type = 'b_plus'
        elif cond == 3:
            trial_type = 'b_minus'
        elif cond == 4: #A
            continue
        elif cond == 5: 
            trial_type = 'c_minus'
        elif cond == 6:
            trial_type = 'c_plus'

        rew = rew_trial[n]

        delta = rew - V[trial_type][-1]
        new_V = V[trial_type][-1] + alpha * delta
        V[trial_type].append(new_V)
    return V